**📘 Notebook 02 — Data Cleaning, Target Engineering & Leakage-Safe Preprocessing**

**1. Scope**

This part prepares a clean, leakage-safe dataset for modelling readmission risk. It focuses on:

- Target variable engineering (30-day readmission)
- Data cleaning (missing values, duplicates, obvious inconsistencies)
- Leakage prevention (removing identifiers and high-risk leakage fields)
- Leakage-safe train/test split (stratified)
- Building a reproducible preprocessing pipeline (imputation + encoding + scaling)
- Saving artefacts required for downstream notebooks (Notebook 03–05)

**Out of scope** for this notebook: model training, hyperparameter tuning, SHAP/LIME, fairness metrics.



This notebook addresses the following capstone stage:

- **Step 3 – Data Preprocessing & Feature Engineering Foundations:**  
  Target engineering (30-day readmission), leakage control, missing value handling,
  stratified/group train-test split, and a reproducible preprocessing pipeline (fit on train only).

This notebook intentionally does **not** cover:
- Detailed EDA and feature selection (Notebook 03)
- Model training, tuning, and evaluation (Notebook 04)
- Explainability and bias/fairness auditing (Notebook 05)
- Final communication and slide decks (Step 6)

Subsequent notebooks will cover:
- **Notebook 03:** Applied EDA, feature selection, and dimensionality reduction (PCA; optional t-SNE/UMAP visualisation)
- **Notebook 04:** Model implementation and comparison (baseline → tuned models)
- **Notebook 05:** Explainability (SHAP/LIME/PDP) + bias/fairness audit and mitigations


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from pathlib import Path

PROJECT_FOLDER = "JohnRaffyRaymundo_AIMCapstone2025"
PROJECT_ROOT = Path("/content/drive/MyDrive") / PROJECT_FOLDER

DATA_DIR = PROJECT_ROOT / "data"
DOCS_DIR = PROJECT_ROOT / "docs"
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"
MODELS_DIR = PROJECT_ROOT / "models"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

# Optional: keep processed data separate
PROCESSED_DIR = OUTPUTS_DIR / "processed"

for d in [DATA_DIR, DOCS_DIR, NOTEBOOKS_DIR, MODELS_DIR, OUTPUTS_DIR, PROCESSED_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("✅ DATA_DIR:", DATA_DIR)
print("✅ PROCESSED_DIR:", PROCESSED_DIR)


✅ PROJECT_ROOT: /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025
✅ DATA_DIR: /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/data
✅ PROCESSED_DIR: /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed


In [3]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 80)

**2. Load Raw Data**

To ensure reproducibility, each notebook reloads the raw dataset independently (no hidden state from previous notebooks).

In [4]:
data_file = DATA_DIR / "diabetic_data.csv"

if not data_file.exists():
    raise FileNotFoundError(
        f"❌ {data_file} not found.\n"
        "Please ensure diabetic_data.csv is placed inside your Google Drive data/ folder."
    )

df_raw = pd.read_csv(data_file)
print("✅ Loaded dataset:", df_raw.shape)
df_raw.head()

✅ Loaded dataset: (101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


**3. Target Engineering (30-Day Readmission)**

The original dataset target column readmitted has three categories:

- <30 : readmitted within 30 days
- >30 : readmitted after 30 days
- NO : no readmission

For this capstone, we define the supervised learning target:

- readmitted_30_days = 1 if readmitted == '<30'
- readmitted_30_days = 0 if readmitted in {'>30', 'NO'}

This aligns with operational use: hospitals typically prioritise interventions within the first 30 days after discharge.

In [5]:
df = df_raw.copy()

if "readmitted" not in df.columns:
    raise KeyError("Column 'readmitted' not found in dataset.")

print("Original target distribution:")
display(df["readmitted"].value_counts(dropna=False))
display((df["readmitted"].value_counts(normalize=True) * 100).round(2))

df["readmitted_30_days"] = (df["readmitted"] == "<30").astype(int)

print("\nBinary target distribution (readmitted_30_days):")
display(df["readmitted_30_days"].value_counts())
display((df["readmitted_30_days"].value_counts(normalize=True) * 100).round(2))


Original target distribution:


,count
readmitted,
NO,54864
>30,35545
<30,11357


,proportion
readmitted,
NO,53.91
>30,34.93
<30,11.16



Binary target distribution (readmitted_30_days):


,count
readmitted_30_days,
0,90409
1,11357


,proportion
readmitted_30_days,
0,88.84
1,11.16


**4. Leakage Control (Critical)**

To avoid data leakage and inflated model performance, we remove fields that:
- Directly identify the encounter/patient (IDs)
- Would not be available at the decision point (discharge planning)
- Could enable memorisation rather than generalisation

At minimum, the following identifiers must be removed:
- encounter_id
- patient_nbr

We also remove the original multiclass readmitted column after deriving our binary target.
Note: Additional leakage risk checks are performed in later notebooks (e.g., SHAP + sanity checks). Here, we apply conservative first-pass leakage prevention.

In [6]:
leakage_cols = []
for c in ["encounter_id", "patient_nbr", "readmitted"]:
    if c in df.columns:
        leakage_cols.append(c)

print("Columns flagged for removal (leakage / identifiers):", leakage_cols)

df = df.drop(columns=leakage_cols)
print("✅ Shape after leakage column removal:", df.shape)

Columns flagged for removal (leakage / identifiers): ['encounter_id', 'patient_nbr', 'readmitted']
✅ Shape after leakage column removal: (101766, 48)


**5. Basic Cleaning**

The dataset uses "?" as a missing value marker in multiple categorical features. We convert "?" to NaN for consistent handling.

We also check duplicates at row level (not expected, but verified).

In [7]:
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

# If duplicates exist, we would drop them. For transparency:
if dup_count > 0:
    df = df.drop_duplicates()
    print("✅ Dropped duplicates. New shape:", df.shape)

Duplicate rows: 0


In [8]:
q_mark_count = (df == "?").sum().sum()
print("Total '?' markers before replacement:", int(q_mark_count))

df = df.replace("?", np.nan)

q_mark_count_after = (df == "?").sum().sum()
print("Total '?' markers after replacement:", int(q_mark_count_after))

Total '?' markers before replacement: 192849
Total '?' markers after replacement: 0


**6. Missingness Report**

Before imputing, we quantify missingness by feature. This supports transparent decisions in preprocessing and later feature selection.

In [9]:
missing_summary = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_pct": (df.isna().mean() * 100).round(2),
    "dtype": df.dtypes.astype(str)
}).sort_values("missing_pct", ascending=False)

missing_summary.head(20)

,missing_count,missing_pct,dtype
weight,98569,96.86,object
max_glu_serum,96420,94.75,object
A1Cresult,84748,83.28,object
medical_specialty,49949,49.08,object
payer_code,40256,39.56,object
race,2273,2.23,object
diag_3,1423,1.40,object
diag_2,358,0.35,object
diag_1,21,0.02,object
time_in_hospital,0,0.00,int64


In [10]:
# Save for documentation
missing_path = DOCS_DIR / "missingness_summary.csv"
missing_summary.to_csv(missing_path, index=True)
print("✅ Saved missingness summary to:", missing_path)

✅ Saved missingness summary to: /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/docs/missingness_summary.csv


**7. Define Features and Target**

We separate input features X from target y.
We also ensure the target has no missing values (it should not, given our construction).

In [11]:
target_col = "readmitted_30_days"

if df[target_col].isna().any():
    raise ValueError("Target contains missing values. Please review target engineering.")

X = df.drop(columns=[target_col])
y = df[target_col].copy()

print("X shape:", X.shape)
print("y shape:", y.shape)
y.value_counts()


X shape: (101766, 47)
y shape: (101766,)


,count
readmitted_30_days,
0,90409
1,11357


**8. Train/Test Split (Stratified)**

We split data after:
- target engineering
- identifier/leakage column removal
- standard missing conversion

We stratify by the binary target because readmission is imbalanced.

Because individual patients may have multiple hospital encounters, a standard random
train–test split risks placing the same patient in both training and test sets.
This would artificially inflate model performance.

To prevent patient-level leakage, a group-based split is performed using `patient_nbr`,
ensuring that all encounters for a given patient appear in only one dataset.


In [12]:
from sklearn.model_selection import GroupShuffleSplit

# We temporarily use patient_nbr ONLY for splitting to avoid patient-level leakage
if "patient_nbr" not in df_raw.columns:
    raise KeyError("patient_nbr not found. Required for leakage-safe group split.")

# Reconstruct dataset to include patient_nbr for grouping
df_split = df_raw.copy().replace("?", np.nan)

# Recreate target
df_split["readmitted_30_days"] = (df_split["readmitted"] == "<30").astype(int)

# Define groups
groups = df_split["patient_nbr"]

# Drop true leakage columns AFTER grouping
df_split = df_split.drop(columns=["encounter_id", "readmitted"])

X = df_split.drop(columns=["readmitted_30_days"])
y = df_split["readmitted_30_days"]

gss = GroupShuffleSplit(
    n_splits=1,
    test_size=0.20,
    random_state=42
)

train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()

# Now remove patient_nbr from features to prevent leakage
X_train = X_train.drop(columns=["patient_nbr"])
X_test = X_test.drop(columns=["patient_nbr"])

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)

print("\nTrain target distribution (%):")
display((y_train.value_counts(normalize=True) * 100).round(2))

print("\nTest target distribution (%):")
display((y_test.value_counts(normalize=True) * 100).round(2))

Train shapes: (81613, 47) (81613,)
Test shapes: (20153, 47) (20153,)

Train target distribution (%):


,proportion
readmitted_30_days,
0,88.72
1,11.28



Test target distribution (%):


,proportion
readmitted_30_days,
0,89.33
1,10.67


**9. Preprocessing Pipeline (Fit on Train Only)**

We build a reproducible preprocessing pipeline using ColumnTransformer:

- Numeric features: median imputation + standard scaling

- Categorical features: most frequent imputation + one-hot encoding

This pipeline is fit only on training data to avoid leakage, then applied to the test set.

We save:

- The fitted preprocessing pipeline (models/preprocess_pipeline.joblib)

- Transformed matrices (outputs/processed/)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Detect types from training set
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X_train.columns if c not in numeric_features]

print("Numeric features:", len(numeric_features))
print("Categorical features:", len(categorical_features))

print("\nSample numeric:", numeric_features[:10])
print("Sample categorical:", categorical_features[:10])


Numeric features: 11
Categorical features: 36

Sample numeric: ['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient']
Sample categorical: ['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum']


In [14]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop"
)

preprocess

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['admission_type_id',
                                  'discharge_disposition_id',
                                  'admission_source_id', 'time_in_hospital',
                                  'num_lab_procedures', 'num_procedures',
                                  'num_medications', 'number_outpatient',
                                  'number_emergency', 'number_inpatient',
                                  'number_diagnoses...
                                  'payer_code', 'medical_specialty', 'diag_1',
                                  'diag_2', 'diag_3', 'max_glu_serum',
                                  'A1Cresult', 'metformin', 'repaglinide',
                                  'nateglinide', 'chlorpropamide',
                                  'glimepiride', 'acetohexamide', 'glipizide',
                                  'glyburide', 'tolbutamide', 'pioglitazone',
                                  'rosiglitazone', 'acarbose', 'miglitol',
                                  'troglitazone', 'tolazamide', 'examide',
                                  'citoglipton', 'insulin',
                                  'glyburide-metformin', ...])])

In [15]:
X_train_processed = preprocess.fit_transform(X_train)
X_test_processed = preprocess.transform(X_test)

print("✅ Processed train shape:", X_train_processed.shape)
print("✅ Processed test shape:", X_test_processed.shape)

✅ Processed train shape: (81613, 2376)
✅ Processed test shape: (20153, 2376)


**10. Save Artefacts**

We save train/test splits and preprocessing artefacts for reproducible downstream modelling.

Notes:

- Since the processed matrices are sparse, we store them efficiently using SciPy .npz.
- We also save the fitted pipeline using Joblib.

In [16]:
# Save raw splits (before transformation) for traceability
X_train_path = PROCESSED_DIR / "X_train_raw.csv"
X_test_path = PROCESSED_DIR / "X_test_raw.csv"
y_train_path = PROCESSED_DIR / "y_train.csv"
y_test_path = PROCESSED_DIR / "y_test.csv"

X_train.to_csv(X_train_path, index=False)
X_test.to_csv(X_test_path, index=False)
y_train.to_csv(y_train_path, index=False)
y_test.to_csv(y_test_path, index=False)

print("✅ Saved raw splits:")
print(" -", X_train_path)
print(" -", X_test_path)
print(" -", y_train_path)
print(" -", y_test_path)

✅ Saved raw splits:
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/X_train_raw.csv
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/X_test_raw.csv
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/y_train.csv
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/y_test.csv


In [17]:
from scipy import sparse
import joblib

# Save sparse matrices
X_train_npz = PROCESSED_DIR / "X_train_processed.npz"
X_test_npz = PROCESSED_DIR / "X_test_processed.npz"

sparse.save_npz(X_train_npz, X_train_processed)
sparse.save_npz(X_test_npz, X_test_processed)

# Save preprocessing pipeline
pipeline_path = MODELS_DIR / "preprocess_pipeline.joblib"
joblib.dump(preprocess, pipeline_path)

print("✅ Saved processed matrices:")
print(" -", X_train_npz)
print(" -", X_test_npz)

print("\n✅ Saved preprocessing pipeline:")
print(" -", pipeline_path)


✅ Saved processed matrices:
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/X_train_processed.npz
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/outputs/processed/X_test_processed.npz

✅ Saved preprocessing pipeline:
 - /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/models/preprocess_pipeline.joblib


In [18]:
# Get output feature names (especially important after OneHotEncoding)
try:
    feature_names = preprocess.get_feature_names_out()
except Exception as e:
    feature_names = None
    print("⚠️ Could not extract feature names:", e)

if feature_names is not None:
    feat_path = DOCS_DIR / "processed_feature_names.txt"
    with open(feat_path, "w") as f:
        for name in feature_names:
            f.write(name + "\n")
    print("✅ Saved processed feature names to:", feat_path)
    print("Example feature names:", feature_names[:20])


✅ Saved processed feature names to: /content/drive/MyDrive/JohnRaffyRaymundo_AIMCapstone2025/docs/processed_feature_names.txt
Example feature names: ['num__admission_type_id' 'num__discharge_disposition_id'
 'num__admission_source_id' 'num__time_in_hospital'
 'num__num_lab_procedures' 'num__num_procedures' 'num__num_medications'
 'num__number_outpatient' 'num__number_emergency' 'num__number_inpatient'
 'num__number_diagnoses' 'cat__race_AfricanAmerican' 'cat__race_Asian'
 'cat__race_Caucasian' 'cat__race_Hispanic' 'cat__race_Other'
 'cat__gender_Female' 'cat__gender_Male' 'cat__gender_Unknown/Invalid'
 'cat__age_[0-10)']


**11. Summary and Transition**

In this notebook, we:

- Defined the binary prediction target readmitted_30_days
- Removed identifier columns to prevent leakage
- Converted dataset missing markers ("?") into NaN
- Produced and saved a missingness report for transparency
- Performed a stratified train/test split
- Built a leakage-safe preprocessing pipeline (fit on train only)
- Saved all key artefacts (raw splits, processed matrices, pipeline, feature names)

Next (Notebook 03): Applied EDA and feature engineering on the training set (and careful comparisons vs test set), including correlation analysis, distribution checks, and feature selection groundwork.